# Assignment 2 - TDT4265
#### Sara L. Ludvigsen and Emma H. Buøen
##### April 2020

## Task 1 - Softmax regression with backpropagation
### 1.a - Backpropagation
In this task we will show that 
$$
w_{ji} := w_{ji} - \alpha \delta_j x_i
$$
and that
$$
\delta_j = f'(z_j) \sum_k w_{kj} \delta_k .
$$

To do this, we first need to define our cost function as:
$$
C(w) = - \frac{1}{N} \sum_{n = 1}^{N} \sum_{k = 1}^{K} y^n_k \ln(\hat{y}^n_k)
$$

We know from the problem description that the activation of hidden unit $j$ as $a_j = f(z_j)$, where f is the sigmoid function. It is also stated that
$$
z_j = \sum_{i=0}^d w_{ji} x_i,
$$
and that $d$ is the dimensionality of the input. 

In the previous assignment, we showed that the update rule for for the weights $w_{kj}$ of the output
layer is
$$
w_{kj} := w_{kj} - \alpha \delta_k a_j, 
$$
where $\delta_k = \frac{\partial C}{\partial z_k} = -(y_k - \hat{y}_k)$.

Calculating the update rule for the weights $w_{ji}$:
$$
w_{ji} := w_{ji} - \alpha \frac{\partial C}{\partial w_{ij}} = w_{ji} - \alpha \overbrace{\frac{\partial C}{\partial z_j}}^{\delta_j} \overbrace{\frac{\partial z_j}{\partial w_{ij}}}^{x_i} = \underline{w_{ji} - \alpha \delta_j x_i}
$$
$$
\delta_j = \frac{\partial C}{\partial z_j} = \frac{\partial C}{\partial a_j}\frac{\partial a_j}{\partial z_j} = \sum_k \overbrace{\frac{\partial C}{\partial z_k}}^{\delta_k}  \overbrace{\frac{\partial z_k}{\partial a_j}}^{w_{kj}} \overbrace{\frac{\partial a_j}{\partial z_j}}^{f'(z_j)} = \underline{f'(z_j) \sum_k w_{kj} \delta_k}
$$


### 1.b -  Vectorize computation
We have this expression for updating the weight from one node in a layer to a node in the next layer.  
$$
w_{ji} := w_{ji} - \alpha \delta_j a_i
$$

Now we have to expand this to updating all the weights from all the nodes in the hidden layer to all the nodes in the output layer, and updating all the weights from the input layer to all the nodes in the hidden layer. 

First from the hidden layer to the output layer:

$$
w_{kj} := w_{kj} - \alpha \delta_k a_j
$$

$$
\delta^K = -(Y_k - \hat{Y}_k)
$$

Where $Y_k$ and $\hat{Y}_k$ are the target output and the computed output respectivly. They are both vectores of dimension ten since we use ten output classes. 

Now for a hidden layer we use the backpropagation error equation we found in $1a$ that support multiple layers:
$$
\delta_j^l = f'(z_j^l) \sum_{k=1}^K w_{kj}^{l+1} \delta_k^{l+1}
$$
In this equation we use $l$ and $l+1$ to implicate which layer the components belong to.
The equation describes the backpropagation error equation for the $j_{th}$ node in layer $l$. The dimensions of $\delta^l$ is $[J,1]$, where $J$ is the number of nodes in layer $l$. The dimensions of $\delta^{l+1}$ is $[K,1]$, where $K$ is the number of nodes in layer $l+1$.

Next, we want to calculate all the backpropagation errors for all the nodes in layer $l$ using matrix multiplication. By giving each $j$ a row in the vector representation, we can describe the weight matrix as $W$ where the weights from node $j$ is represented in row $j$ in $W$. Similarily, we create the matrix $\Gamma'(z^l)$ with the values of $f'(z_j^l)$ along the diagonal, and all other values set to zero. 
The dimensions of $W$ is $[J,K]$, and the dimensions of $J$ is $[J,J]$

Our equation for the backpropagation error now look like this:
$$
\delta^l =  \Gamma'(z^l)(W^{l+1})\delta^{l+1}
$$

The new updating law is represented like this:
$$
W^l := W^l - \alpha \delta^l (a^{l-1})^{\top}
$$

The updating law for the weights from the hidden layer to the output layer is represented as follows:
$$
W^k := W^k + \alpha \delta^K (a^{j}) = W^k + \alpha (Y_k - \hat{Y}_k) (a^{j})
$$

And the updating law for the weights from the input layer to the hidden layer is represented as:
$$
W^j := W^j - \alpha \delta^j (a^{i}) = W^j - \alpha \Gamma'(z^j)(W^{k})\delta^{K} (a^{i}) = W^j + \alpha \Gamma'(z^j)(W^{k})(Y_k - \hat{Y}_k)(a^{i})
$$

This is easy enough to implement in python using numpy.

## Task 2 - Softmax Regression with Backpropagation
### 2.a - Preprocessing
This is how we implemented the `pre_process_images` function. We modified it to have mean and standard deviation as parameters.

```python
def pre_process_images(X: np.ndarray, mean: float, std:float):
    """
    Args:
        X: images of shape [batch size, 784] in the range (0, 255)
    Returns:
        X: images of shape [batch size, 785]
    """
    assert X.shape[1] == 784,\
        f"X.shape[1]: {X.shape[1]}, should be 784"

    X = np.insert(X, 0, 1, axis=1)
    
    X = (X-(mean))/(std)
    X = np.divide(X,255)
    return X
```

We calculate the mean and std in `main` in `task2a.py`:
```python
    mean = np.mean(X_train)
    std = np.std(X_train)
    X_train = pre_process_images(X_train, mean, std)
```

This is how it processes images in `task2.py`, in its `main`function:
```python
    X_train, Y_train, X_val, Y_val, X_test, Y_test = utils.load_full_mnist(validation_percentage)
    mean = np.mean(X_train)
    std = np.std(X_train)
    X_train = pre_process_images(X_train, mean, std)
    X_test = pre_process_images(X_test, mean, std)
    X_val = pre_process_images(X_val, mean, std)
    Y_train = one_hot_encode(Y_train, 10)
    Y_val = one_hot_encode(Y_val, 10)
    Y_test = one_hot_encode(Y_test, 10)
```

### 2.b - Softmax model
The following code shows our implementation of `forward` and `backward` in the file `task2a.py`.

```python
def sigmoid(x: np.ndarray):
    return 1 / (1 + np.exp(-x))

def softmax(a: np.ndarray):
    a_exp = np.exp(a)
    return a_exp / a_exp.sum(axis=1, keepdims=True)
    
class SoftmaxModel:

    def __init__(self,
                 # Number of neurons per layer
                 neurons_per_layer: typing.List[int],
                 use_improved_sigmoid: bool,  # Task 3a hyperparameter
                 use_improved_weight_init: bool  # Task 3c hyperparameter
                 ):
        # Define number of input nodes
        self.I = 785
        self.use_improved_sigmoid = use_improved_sigmoid

        # Define number of output nodes
        # neurons_per_layer = [64, 10] indicates that we will have two layers:
        # A hidden layer with 64 neurons and a output layer with 10 neurons.
        self.neurons_per_layer = neurons_per_layer

        # Initialize the weights
        self.ws = []
        prev = self.I
        for size in self.neurons_per_layer:
            w_shape = (prev, size)
            print("Initializing weight to shape:", w_shape)
            w = np.random.uniform(-1,1,(w_shape))            
            self.ws.append(w)
            prev = size
        self.grads = [0 for i in range(len(self.ws))]

        #define a_i and a_j
        self.z_j = []
        self.a_j = []

    def forward(self, X: np.ndarray) -> np.ndarray:
        """
        Args:
            X: images of shape [batch size, 785]
        Returns:
            y: output of model with shape [batch size, num_outputs]
        """
        self.z_j = np.matmul(X,self.ws[0]) #(100,64)
        self.a_j = sigmoid(self.z_j) #(100,64)
        z_k = np.matmul(self.a_j,self.ws[1]) #(100,10)
        output = softmax(z_k) #(100,10)
        return output

    def backward(self, X: np.ndarray, outputs: np.ndarray,
                 targets: np.ndarray) -> None:
        """
        Args:
            X: images of shape [batch size, 785]
            outputs: outputs of model of shape: [batch size, num_outputs]
            targets: labels/targets of each image of shape: [batch size, num_classes]
        """
        assert targets.shape == outputs.shape,\
            f"Output shape: {outputs.shape}, targets: {targets.shape}"
        # A list of gradients.
        # For example, self.grads[0] will be the gradient for the first hidden layer

        d_k = -(targets - outputs) #(100x10)
        grad_1 = np.matmul(self.a_j.T, d_k)

        df = self.a_j*(1-self.a_j) #(100x64)
        temp = self.ws[1].dot(d_k.T) #(64x100)
        d_j = df.T*temp #(64x100)
        grad_0 = np.matmul(X.T, d_j.T)

        self.grads = [grad_0, grad_1]


        for grad, w in zip(self.grads, self.ws):
            assert grad.shape == w.shape,\
                f"Expected the same shape. Grad shape: {grad.shape}, w: {w.shape}."
```

### 2.c - Training
![alt text](softmax_train_graph_2.png "2c")
The plot to the left is the validation and training loss for every gradient step. The right plot is the training and validation accuracy. 

### 2.d - Parameters
The number of parameters in our network is equal to the number of weights + the number of biases.
$$number of parameters = 785*64 + 64*10 = 50 880$$

## Task 3 - "Tricks of the trade"
Without "Tricks of the trade"

Metric | Value
--- | ---
Final Train Cross Entropy Loss| 0.031105040640134807
Final Validation Cross Entropy Loss| 0.03486005051888923
Final Test Cross Entropy Loss| 0.03211263794100177
Final Train accuracy| 0.8949375
Final Validation accuracy| 0.8888333333333334
Final Test accuracy| 0.8932



### 3.a - Shuffling
We implemented "Shuffling" as follows:
```python
def shuffle_in_unison(a, b):
    state = np.random.get_state()
    a = np.random.shuffle(a)
    np.random.set_state(state)
    b = np.random.shuffle(b)
    return a,b
```
With "shuffling", we got the following results:

Metric | Value
--- | ---
Final Train Cross Entropy Loss| 0.02755075691377316
Final Validation Cross Entropy Loss| 0.031061759331951252
Final Test Cross Entropy Loss| 0.028824543997236746
Final Train accuracy| 0.9101666666666667
Final Validation accuracy| 0.8984166666666666
Final Test accuracy| 0.9062

We can observe a noticeable improvement in the test loss and all the accuracies. 
![alt text](A2_3a.png "3a")

### 3.b - Improved Sigmoid
This is our implementation of the improved sigmoid function:
```python
def improved_sigmoid(x: np.ndarray):
    return 1.7159*np.tanh((2/3)*x)

def improved_sigmoid_derivative(x: np.ndarray):
    return 1.7159*2.0 / (3.0*(np.cosh(((2.0/3.0)*x))**2))
```

We got the following results:

Metric | Value
--- | ---
Final Train Cross Entropy Loss| 0.02435081307283175
Final Validation Cross Entropy Loss| 0.030451398557699797
Final Test Cross Entropy Loss| 0.02835055746990178
Final Train accuracy| 0.9205416666666667
Final Validation accuracy| 0.90725
Final Test accuracy| 0.9127

![alt text](A2_3b.png "3b")

### 3.c - Improved weight initialization
The code in `task2a.py` where we initialize the weights:
```python
# Initialize the weights
self.ws = []
prev = self.I
for size in self.neurons_per_layer:
    w_shape = (prev, size)
    print("Initializing weight to shape:", w_shape)
    if use_improved_weight_init == False:
        w = np.random.uniform(-1,1,(w_shape))        
    else: # Improved weight init
        w = np.random.normal(0, 1/(np.sqrt(w_shape[1])), w_shape)    
    self.ws.append(w)
    prev = size
self.grads = [0 for i in range(len(self.ws))]
```

Metric | Value
--- | ---
Final Train Cross Entropy Loss| 0.030247335380772573
Final Validation Cross Entropy Loss| 0.03377519645466785
Final Test Cross Entropy Loss| 0.03118569270086158
Final Train accuracy| 0.89775
Final Validation accuracy| 0.8916666666666667
Final Test accuracy| 0.8955

![alt text](A2_3c.png "3c")

### 3.d - Momentum
In the `train()` function i `task2.py`:
```python
    momentum = [0 for i in range(len(model.grads))]

    global_step = 0
    for epoch in range(num_epochs):
        # Shuffling before next epoch
        if use_shuffle == True:
            shuffle_in_unison(X_train, Y_train)
        for step in range(num_batches_per_epoch):
            start = step * batch_size
            end = start + batch_size
            X_batch, Y_batch = X_train[start:end], Y_train[start:end]

            y_hat = model.forward(X_batch)
            model.backward(X_batch, y_hat, Y_batch)

            if use_momentum == True:
                momentum[0] = (1-momentum_gamma)*model.grads[0] + momentum_gamma*momentum[0]
                momentum[1] = (1-momentum_gamma)*model.grads[1] + momentum_gamma*momentum[1]
                model.ws[0] += -1*learning_rate*(momentum[0])
                model.ws[1] += -1*learning_rate*(momentum[1])

                '''
                momentum(k+1) = gamma*momentum(k) + (1-gamma)*model.grads(k)
                '''
            else:
                model.ws[0] += -1*learning_rate*model.grads[0]
                model.ws[1] += -1*learning_rate*model.grads[1]
```

We got the following results with momentum:

Metric | Value
--- | ---
Final Train Cross Entropy Loss| 0.028527996874256067
Final Validation Cross Entropy Loss| 0.031918324737347306
Final Test Cross Entropy Loss| 0.029540150765479535
Final Train accuracy| 0.9071041666666667
Final Validation accuracy| 0.8990833333333333
Final Test accuracy| 0.9051

![alt text](A2_3d.png "3d")

### 3.e - Comparision
If we use all the "tricks of the trade", we get these results:

Metric | Value
--- | ---
Final Train Cross Entropy Loss| 0.024069764656181148
Final Validation Cross Entropy Loss| 0.02961587176035768
Final Test Cross Entropy Loss| 0.0275746366585815
Final Train accuracy| 0.9216458333333334
Final Validation accuracy| 0.9075
Final Test accuracy| 0.9151

![alt text](A2_3e.png "3e")

#### Comparision, rounded to four decimals:

Metric | Without tricks | With shuffling |Improved sigmoid|Improved weights | Momentum|All improvements
--- | --- | ---| ---| ---| ---| ---
Final Train Cross Entropy Loss| 0.0311|0.0276|0.0244|0.0302|0.0285|0.0241
Final Validation Cross Entropy Loss| 0.0349|0.0311|0.0305|0.0338|0.0319|0.0296
Final Test Cross Entropy Loss| 0.0321|0.0288|0.0284|0.0312|0.0295|0.0276
Final Train accuracy| 0.8949|0.9102|0.9205|0.8978|0.9071|0.9216
Final Validation accuracy| 0.8888|0.8984|0.9073|0.8917|0.8991|0.9075
Final Test accuracy| 0.8932|0.9062|0.9127|0.8955|0.9051|0.9151


